# Titanic Competition

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

import os

from utils import get_variable_name, get_num_missing_values, get_categorical_columns

## Read and display the data

In [2]:
# Read the data
train_data = pd.read_csv("../input/train.csv").set_index("PassengerId")
test_data = pd.read_csv("../input/test.csv").set_index("PassengerId")

In [3]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
X = train_data.copy()
X_test = test_data.copy()

# Drop rows with empty label value
X.dropna(axis=0, subset=['Survived'], inplace=True)
y = X.Survived
X.drop(['Survived'], axis=1, inplace=True)

In [6]:
# What can be definitely dropped?
# Name, Ticket
X.drop(['Name', 'Ticket'], axis=1, inplace=True)
X_test.drop(['Name', 'Ticket'], axis=1, inplace=True)

In [7]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,
1,3,male,22.0,1,0,7.2500,NaN,S
2,1,female,38.0,1,0,71.2833,C85,C
3,3,female,26.0,0,0,7.9250,NaN,S
4,1,female,35.0,1,0,53.1000,C123,S
5,3,male,35.0,0,0,8.0500,NaN,S


In [8]:
get_num_missing_values(X)

Missing values in variable columns:
Age         177
Cabin       687
Embarked      2
dtype: int64


In [9]:
get_categorical_columns(X)

variable columns with categorical data:
[('Sex', 2), ('Embarked', 3), ('Cabin', 147)]


In [10]:
get_num_missing_values(X_test)

Missing values in variable columns:
Age       86
Fare       1
Cabin    327
dtype: int64


In [11]:
get_categorical_columns(X_test)

variable columns with categorical data:
[('Sex', 2), ('Embarked', 3), ('Cabin', 76)]


In [12]:
# With so many Cabin entries missing, the Cabin column can also be removed
X.drop(['Cabin'], axis=1, inplace=True)
X_test.drop(['Cabin'], axis=1, inplace=True)

In [13]:
# Select categorical columns
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
# Select numerical columns
numerical_cols = [
    cname for cname in X.columns if X[cname].dtype in ["int64", "float64"]
]
# Keep selected columns only
my_cols = categorical_cols + numerical_cols

In [14]:
print(categorical_cols)
print(numerical_cols)
print(my_cols)

['Sex', 'Embarked']
['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
['Sex', 'Embarked', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [15]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

My research on ML models among which were:
- Decision Tree Classifier
- Random Forest Classifier
- XGB Classifier
- Logistic Regression Classifier
- Support Vector Machine
- Gradient Boosting Classifier
Proved that the best model was GradientBoostingClassifier(criterion='friedman_mse', n_estimators=400, learning_rate=0.06, loss=log_loss) giving the average validation accuracy of 83.6%.
I also considered the inclusion of 'Age' column and even though about 25% of the age data was missing the results were better.

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

# Define a model
model = GradientBoostingClassifier(
    criterion="friedman_mse",
    n_estimators=400,
    learning_rate=0.06,
    loss="log_loss",
    random_state=0,
)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

# Preprocessing of training data, fit model
my_pipeline.fit(X, y)

# Preprocessing of validation data, get predictions
predictions = my_pipeline.predict(X_test)

# Saving the predictions
output = pd.DataFrame({"PassengerId": X_test.index, "Survived": predictions})

if not os.path.isdir("output/"):
    os.mkdir("output/")
    
output.to_csv("output/submission.csv", index=False)

print("Your submission was successfully saved!")

Your submission was successfully saved!
